## Code Hist.

 - CODE : 단지 내 동-층-세대 Feature('HOUSE_ID_HO_INT') 생성  
  - DESC : int_Domain의 변경에 따라, 아래와 같은 작업을 수행  
  &emsp;1) 에너지 사용량 Raw 데이터로부터 Target Feature 생성  
  &emsp;2) 생성된 Feature를 기존 Raw Data에 반영 및 사본 저장  
  &emsp;3) 각 Domain별 공통 'HOUSE_ID_HO_INT'를 도출, hList.csv로 저장

 - DATE  
    &ensp; 2024-01-25 Created  
    &ensp; 2024-07-23 코드 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 코드 간략화  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 전체 도메인에 대한 수행 부분 추가  
    &ensp; 2024-07-25 프로세스 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 고유 동-층-세대 Feature를 생성 및 사용  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 각 Domain별 공통 hList 생성 및 사용  

## 01. Code

### 01. Init

#### 01-01. Init_Module Import

In [7]:
#region Import_Basic Module
## Basic
import os, sys, warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('./__file__'))
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('./__file__'))))
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series
pd.options.display.float_format = '{:.10f}'.format

import math, random

## Datetime
import time, datetime as dt
from datetime import datetime, date, timedelta

## glob
import glob, requests, json
from glob import glob

## 시각화
import matplotlib.pyplot as plt, seaborn as sns
# %matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## Excel/CSV
import openpyxl, xlrd

## Split, 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# CLustering 알고리즘의 성능 평가 측도
from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## 정규화
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics

## For Web
import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm
import tqdm
#endregion Import_Basic Module

In [8]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch, torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf, tensorflow_addons as tfa
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential, load_model
    from keras_flops import get_flops
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [9]:
## Import_Local Module
from Src_Dev_Common import Data_Datetime as com_date, KMA_Weather as com_KMA, KECO_AirKor as com_KECO, KASI_Holiday as com_Holi, KIER_Usage_M02 as com_KIER_M02

#### 01-02. Config (Directory, Params)

In [10]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"], os.environ['TF_DETERMINISTIC_OPS'] = str(SEED), "1"

In [11]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-07-26 16:33:58.738556
2024 / 7 / 26
16 : 33


### 02. Create Feature : HOUSE_ID

#### 02-01. 사전 확인 및 Domain별 공통 세대 추출
1) Raw Dataset은 단지에 존재하는 동('HOUSE_ID_DONG') / 층('HOUSE_ID_HO_PRE') / 호실('HOUSE_ID_HO')별  
에너지 사용량으로 구성

2) 에너지 Domain은 총 5가지(전력/난방/수도/온수/가스)로 구성되어있으나, 온수 데이터는 유량과 열량으로 분리 수집됨.  
동/층은 Int값으로, 호실은 40~50자리에 이르는 Hash값으로 구성

3) 본 코드의 목적은 동-층-세대에 대한 고유 ID를 나타내는 Feature를 생성, 3개 Feature를 1개 Feature로 압축하는 것.  
이를 위해 각 동/층/세대가 동일하게 구성되어있는지 확인해야함  
(곧, 각 Domain별 사용량이 측정된 세대의 구성이 일정한지 확인해야함.) 

##### Case 01. KIER_hList_Common이 존재하지 않을 때  
공통 hList가 추출되지 않은 경우

In [12]:
df_kier_hList_com = pd.DataFrame()
for i in range(0, 6):
    ## ▶ Define Variable and Directories
    ## Dict_Domain
    ## dict_domain = {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 5:"GAS"} ## GAS는 사용하지 않음.
    int_domain = i
    ## Domain, ACCU/INST Column
    str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
    ## Directory Root
    str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)
    ## Interval, Target File
    str_interval, str_fileRaw, str_file_hList, str_file = com_KIER_M02.create_file_str(str_domain)

    print("■ " + str_domain)
    df_kier_hList_tmp = pd.read_csv(str_dir_raw + str_fileRaw, index_col = 0, error_bad_lines = False).drop_duplicates(subset = ['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO']).reset_index()[['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO']]

    ## "KeyError : 'HOUSE_ID_DONG'" 방지 : 최초 df_kier_hList_com는 빈 DataFrame이므로
    if i == 0 : df_kier_hList_com = df_kier_hList_tmp
    else : df_kier_hList_com = pd.merge(df_kier_hList_com, df_kier_hList_tmp, how = 'inner', on = ['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO'])

print(df_kier_hList_com.shape, ' /// ', df_kier_hList_com.columns)
str_file_hList = str('KIER_hList_Common.csv')
df_kier_hList_com.to_csv(str_dir_raw + str_file_hList)

0 : ELEC
str_fileRaw : KIER_RAW_ELEC_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_ELEC_2024-06-07.csv
str_file : KIER_ELEC_INST_0_0.csv
■ ELEC
1 : HEAT
str_fileRaw : KIER_RAW_HEAT_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_HEAT_2024-06-07.csv
str_file : KIER_HEAT_INST_0_0.csv
■ HEAT
2 : WATER
str_fileRaw : KIER_RAW_WATER_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_WATER_2024-06-07.csv
str_file : KIER_WATER_INST_0_0.csv
■ WATER
3 : HOT_HEAT
str_fileRaw : KIER_RAW_HOT_HEAT_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_HOT_HEAT_2024-06-07.csv
str_file : KIER_HOT_HEAT_INST_0_0.csv
■ HOT_HEAT
4 : HOT_FLOW
str_fileRaw : KIER_RAW_HOT_FLOW_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_HOT_FLOW_2024-06-07.csv
str_file : KIER_HOT_FLOW_INST_0_0.csv
■ HOT_FLOW
5 : GAS
str_fileRaw : KIER_RAW_GAS_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_GAS_2024-06-07.csv
str_file : KIER_GAS_INST_0_0.csv
■ GAS


Skipping line 52380994: expected 5 fields, saw 6



(348, 3)  ///  Index(['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO'], dtype='object')


##### Case 02. KIER_hList_Common이 존재할 때  

In [13]:
str_file_hList = str('KIER_hList_Common.csv')
df_kier_hList_com = pd.read_csv(str_dir_raw + str_file_hList, index_col = 0)[['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO']]
df_kier_hList_com

,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO
0,561,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=
1,561,1,ODBEOTE0QzZCMjg5RUMzNTRBQjM1NjE0RDNBNDE0NzU=
2,561,1,QTUwRUJDQTlBMkZENUQyNjg0RDRGREVFMTNGNDI3NUY=
3,561,1,RkRCQzY4OTlCMTRDQjgzNUZBREE3Njc2MkM2MUM2QjI=
4,561,2,MTQxNkQ1RDAzOTE5NTM5RUM3MzlDQkY3RTI2NERCQTM=
...,...,...,...
343,563,23,QkJCQjQwOThFNjgwNjUwQzJGRUJFNDlCNTREOEUxNDU=
344,563,23,QUJENzFFMEFDRjM4MjY3NTRDQzJCRjdDNUQ3ODA0N0Y=
345,563,23,RUVENzZDNTc4MUY4NDg5MUREQjZEOUM1M0Q2RjYzMDE=
346,563,24,NEVENDc1OTFFNDBBRjBDQzkzMTlEMDgzMkMzMkZFQTg=


#### 02-02. 세대별 고유 Feature 생성

In [14]:
## ▶ Create Feature
## Target 변수('HOUSE_ID') 생성을 위한 호실 변수('HOUSE_ID_HO_INT') Column 생성
df_kier_hList_com['HOUSE_ID'], df_kier_hList_com['HOUSE_ID_HO_INT'] = 0, 0
## List : Building
list_Bld = df_kier_hList_com[['HOUSE_ID_DONG']]['HOUSE_ID_DONG'].drop_duplicates()
for bld in list_Bld:
    ## List : Floor
    list_F = df_kier_hList_com[df_kier_hList_com['HOUSE_ID_DONG'] == bld]['HOUSE_ID_HO_PRE'].drop_duplicates()
    
    for f in list_F:
        ## List : House
        list_H = df_kier_hList_com[(df_kier_hList_com['HOUSE_ID_DONG'] == bld)
                                   & (df_kier_hList_com['HOUSE_ID_HO_PRE'] == f)]['HOUSE_ID_HO'].drop_duplicates()

        ## 'HOUSE_ID_HO'는 해시코드 형태로 나타나므로  
        ## 숫자를 부여하여 호실로서 임의재정의 및 hList 내에 저장
        int_house_num = 1
        for h in list_H:
            idx_h = df_kier_hList_com[(df_kier_hList_com['HOUSE_ID_DONG'] == bld)
                                      & (df_kier_hList_com['HOUSE_ID_HO_PRE'] == f)
                                      & (df_kier_hList_com['HOUSE_ID_HO'] == h)].index
            df_kier_hList_com['HOUSE_ID_HO_INT'].loc[idx_h] = int_house_num

            int_house_num = int_house_num + 1

## 'HOUSE_ID' 변수 생성
df_kier_hList_com['HOUSE_ID'] = df_kier_hList_com.apply(lambda row : '-'.join(map(str, [row.HOUSE_ID_DONG, row.HOUSE_ID_HO_PRE, row.HOUSE_ID_HO_INT])), axis = 1)

## 기존 hList에 HOUSE_ID가 취합된 DataFrame을 덮어 씌움
df_kier_hList_com.to_csv(str_dir_raw + str_file_hList)
df_kier_hList_com

,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO,HOUSE_ID,HOUSE_ID_HO_INT
0,561,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,561-1-1,1
1,561,1,ODBEOTE0QzZCMjg5RUMzNTRBQjM1NjE0RDNBNDE0NzU=,561-1-2,2
2,561,1,QTUwRUJDQTlBMkZENUQyNjg0RDRGREVFMTNGNDI3NUY=,561-1-3,3
3,561,1,RkRCQzY4OTlCMTRDQjgzNUZBREE3Njc2MkM2MUM2QjI=,561-1-4,4
4,561,2,MTQxNkQ1RDAzOTE5NTM5RUM3MzlDQkY3RTI2NERCQTM=,561-2-1,1
...,...,...,...,...,...
343,563,23,QkJCQjQwOThFNjgwNjUwQzJGRUJFNDlCNTREOEUxNDU=,563-23-2,2
344,563,23,QUJENzFFMEFDRjM4MjY3NTRDQzJCRjdDNUQ3ODA0N0Y=,563-23-3,3
345,563,23,RUVENzZDNTc4MUY4NDg5MUREQjZEOUM1M0Q2RjYzMDE=,563-23-4,4
346,563,24,NEVENDc1OTFFNDBBRjBDQzkzMTlEMDgzMkMzMkZFQTg=,563-24-1,1


### 03. Adopt Feature : HOUSE_ID

#### Case 01. 각 Domain에 대한 실행

In [15]:
## ▶ Define Variable and Directories
## Dict_Domain
## dict_domain = {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 5:"GAS"} ## GAS는 사용하지 않음.
int_domain = 0
## Domain, ACCU/INST Column
str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
## Directory Root
str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)
## Interval, Target File
str_interval, str_fileRaw, str_file_hList, str_file = com_KIER_M02.create_file_str(str_domain)

0 : ELEC
str_fileRaw : KIER_RAW_ELEC_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_ELEC_2024-06-07.csv
str_file : KIER_ELEC_INST_0_0.csv


In [16]:
## ▶ Load Raw Data
## Gas의 경우는 ParserError: Error tokenizing data 발생하므로 아래와 같이 Load
## 단, warn_bad_lines는 True로 그대로 두어 어느 행이 사라졌는지 확인해야만 함
str_file_hList = str('KIER_hList_Common.csv')
df_kier_hList_com = pd.read_csv(str_dir_raw + str_file_hList, index_col = 0)
df_kier_raw = pd.read_csv(str_dir_raw + str_fileRaw, index_col= 0, error_bad_lines = False).reset_index()

## df_kier_raw와 hList를 Merge 및 유효 Column 정리
## .dropna() 필요 : Gas의 경우, 세대 정보가 유효하지 않은 Data가 1,800건 이상 존재
df_kier_raw = pd.merge(df_kier_raw, df_kier_hList_com, how = 'left', on = ['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO'])[['METER_DATE', 'HOUSE_ID', str_col_accu]].dropna()

print(df_kier_raw.shape, ' /// ', df_kier_raw.columns)
df_kier_raw

(33837156, 3)  ///  Index(['METER_DATE', 'HOUSE_ID', 'ELEC_ACTUAL_ACCU_EFF'], dtype='object')


,METER_DATE,HOUSE_ID,ELEC_ACTUAL_ACCU_EFF
0,2022-07-17 23:20:01,561-1-1,798.6900000000
1,2022-07-17 23:30:01,561-1-1,798.8600000000
2,2022-07-17 23:40:01,561-1-1,799.0200000000
3,2022-07-17 23:50:01,561-1-1,799.1600000000
4,2022-07-18 00:00:01,561-1-1,799.3000000000
...,...,...,...
33837151,2024-06-05 14:00:00,563-24-2,7788.6400000000
33837152,2024-06-05 14:10:00,563-24-2,7788.6800000000
33837153,2024-06-05 14:30:00,563-24-2,7788.7300000000
33837154,2024-06-05 14:40:00,563-24-2,7788.7600000000


In [17]:
## Export df_kier_raw
str_file = str('KIER_RAW_' + str_domain + '_H_ID_Adopted.csv')
df_kier_raw.to_csv(str_dir_raw + str_file)

#### Case 02. 모든 Domain에 대한 실행

In [18]:
for i in range(0, 6):
    ## ▶ Define Variable and Directories
    ## Dict_Domain
    dict_domain = {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 5:"GAS"} ## GAS는 사용하지 않음.
    int_domain = i
    ## Domain, ACCU/INST Column
    str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
    ## Directory Root
    str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)
    ## Interval, Target File
    str_interval, str_fileRaw, str_file_hList, str_file = com_KIER_M02.create_file_str(str_domain)
    
    print("■ " + str_domain)
    ## ▶ Load Raw Data
    ## Gas의 경우는 ParserError: Error tokenizing data 발생하므로 아래와 같이 Load
    ## 단, warn_bad_lines는 True로 그대로 두어 어느 행이 사라졌는지 확인해야만 함
    str_file_hList = str('KIER_hList_Common.csv')
    df_kier_hList_com = pd.read_csv(str_dir_raw + str_file_hList, index_col = 0)
    df_kier_raw = pd.read_csv(str_dir_raw + str_fileRaw, index_col= 0, error_bad_lines = False).reset_index()

    ## df_kier_raw와 hList를 Merge 및 유효 Column 정리
    ## .dropna() 필요 : Gas의 경우, 세대 정보가 유효하지 않은 Data가 1,800건 이상 존재
    df_kier_raw = pd.merge(df_kier_raw, df_kier_hList_com, how = 'left', on = ['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO'])[['METER_DATE', 'HOUSE_ID', str_col_accu]].dropna()

    ## Export df_kier_raw
    str_file = str('KIER_RAW_' + str_domain + '_H_ID_Adopted.csv')
    df_kier_raw.to_csv(str_dir_raw + str_file)

    print(df_kier_raw.shape, ' /// ', df_kier_raw.columns)
    print(df_kier_raw.isna().sum())
df_kier_raw

0 : ELEC
str_fileRaw : KIER_RAW_ELEC_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_ELEC_2024-06-07.csv
str_file : KIER_ELEC_INST_0_0.csv
■ ELEC
(33837156, 3)  ///  Index(['METER_DATE', 'HOUSE_ID', 'ELEC_ACTUAL_ACCU_EFF'], dtype='object')
METER_DATE              0
HOUSE_ID                0
ELEC_ACTUAL_ACCU_EFF    0
dtype: int64
1 : HEAT
str_fileRaw : KIER_RAW_HEAT_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_HEAT_2024-06-07.csv
str_file : KIER_HEAT_INST_0_0.csv
■ HEAT
(27633960, 3)  ///  Index(['METER_DATE', 'HOUSE_ID', 'HEAT_ACCU_HEAT'], dtype='object')
METER_DATE        0
HOUSE_ID          0
HEAT_ACCU_HEAT    0
dtype: int64
2 : WATER
str_fileRaw : KIER_RAW_WATER_2024-06-07.csv
str_fileRaw_hList : KIER_RAW_WATER_2024-06-07.csv
str_file : KIER_WATER_INST_0_0.csv
■ WATER
(32868970, 3)  ///  Index(['METER_DATE', 'HOUSE_ID', 'WATER_ACCU_FLOW'], dtype='object')
METER_DATE         0
HOUSE_ID           0
WATER_ACCU_FLOW    0
dtype: int64
3 : HOT_HEAT
str_fileRaw : KIER_RAW_HOT_HEAT_2024-06-07.cs

Skipping line 52380994: expected 5 fields, saw 6



(52379163, 3)  ///  Index(['METER_DATE', 'HOUSE_ID', 'GAS_ACCU_FLOW'], dtype='object')
METER_DATE       0
HOUSE_ID         0
GAS_ACCU_FLOW    0
dtype: int64


,METER_DATE,HOUSE_ID,GAS_ACCU_FLOW
823,2022-08-10 15:30:09,561-1-1,0.0000000000
824,2022-08-24 10:00:26,561-1-1,0.0000000000
825,2022-09-19 23:54:11,561-1-1,0.1200000000
826,2022-09-20 19:14:12,561-1-1,0.1200000000
827,2022-09-20 20:14:12,561-1-1,0.1200000000
...,...,...,...
52380804,2024-06-24 09:36:59,563-24-2,0.1100000000
52380805,2024-06-24 09:46:59,563-24-2,0.1100000000
52380806,2024-06-24 09:56:59,563-24-2,0.1100000000
52380807,2024-06-24 10:06:59,563-24-2,0.1100000000


In [19]:
df_kier_raw.columns

Index(['METER_DATE', 'HOUSE_ID', 'GAS_ACCU_FLOW'], dtype='object')

### [BU] Old Code 저장

In [20]:
# ## HOT_HEAT와 HOT_FLOW가 한 시트에 존재하는 경우
# str_fileRaw = 'KIER_RAW_HOT_2024-03-13.csv'
# df_hot_tmp = pd.read_csv(str_dir_raw + str_fileRaw, index_col = 0).reset_index()
# df_hot_tmp[['METER_DATE', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO', 'HOT_ACCU_HEAT']].to_csv(str_dir_raw + 'KIER_RAW_HOT_HEAT_2024-03-13.csv')
# df_hot_tmp[['METER_DATE', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO', 'HOT_ACCU_FLOW']].to_csv(str_dir_raw + 'KIER_RAW_HOT_FLOW_2024-03-13.csv')

In [21]:
## [2024-03-13][미사용] HOT Flow와 Heat를 Raw 단계에서 분리해서 사용
## [필요시] HOT_Cleansed.csv를 분리하자!
# str_fileCleansed = str('KIER_RAW_HOT_Cleansed.csv')
# df_raw = pd.read_csv(str_dir_raw + str_fileCleansed
#                      , index_col = 0)

# str_file_hot_heat = str('KIER_RAW_HOT_HEAT_Cleansed.csv')
# df_hot_heat = df_raw[['METER_DATE', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_INT', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO', 'HOT_ACCU_HEAT']]
# df_hot_heat = df_hot_heat.rename(columns = {'HOT_ACCU_HEAT' : 'HOT_HEAT_ACCU'})
# df_hot_heat.to_csv(str_dir_raw + str_file_hot_heat)

# str_file_hot_flow = str('KIER_RAW_HOT_FLOW_Cleansed.csv')
# df_hot_flow = df_raw[['METER_DATE', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_INT', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO', 'HOT_ACCU_FLOW']]
# df_hot_heat = df_hot_heat.rename(columns = {'HOT_ACCU_HEAT' : 'HOT_FLOW_ACCU'})
# df_hot_heat.to_csv(str_dir_raw + str_file_hot_flow)